# Imports

In [1]:
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

# Read CSV data

In [2]:
df2 = pd.read_csv('toddlerData.csv',na_values='?')
df2.head()

,Case_No,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,Age_Mons,Qchat-10-Score,Sex,Ethnicity,Jaundice,Family_mem_with_ASD,Who completed the test,Class/ASD Traits
0,1,0,0,0,0,0,0,1,1,0,1,28,3,f,middle eastern,yes,no,family member,No
1,2,1,1,0,0,0,1,1,0,0,0,36,4,m,White European,yes,no,family member,Yes
2,3,1,0,0,0,0,0,1,1,0,1,36,4,m,middle eastern,yes,no,family member,Yes
3,4,1,1,1,1,1,1,1,1,1,1,24,10,m,Hispanic,no,no,family member,Yes
4,5,1,1,0,1,1,1,1,1,1,1,20,9,f,White European,no,yes,family member,Yes


# Get dummies

In [3]:
within24_36= pd.get_dummies(df2['Age_Mons']>24,drop_first=True)
within0_12 = pd.get_dummies(df2['Age_Mons']<13,drop_first=True)
male=pd.get_dummies(df2['Sex'],drop_first=True)
ethnics=pd.get_dummies(df2['Ethnicity'],drop_first=True)
jaundice=pd.get_dummies(df2['Jaundice'],drop_first=True)
ASD_genes=pd.get_dummies(df2['Family_mem_with_ASD'],drop_first=True)
ASD_traits=pd.get_dummies(df2['Class/ASD Traits '],drop_first=True)

# Generate Columns

In [4]:
final_data= pd.concat([within0_12,within24_36,male,ethnics,jaundice,ASD_genes,ASD_traits],axis=1)
final_data.columns=['within0_12','within24_36','male','Latino','Native Indian','Others','Pacifica','White European','asian','black','middle eastern','mixed','south asian','jaundice','ASD_genes','ASD_traits']
final_data.head()

,within0_12,within24_36,male,Latino,Native Indian,Others,Pacifica,White European,asian,black,middle eastern,mixed,south asian,jaundice,ASD_genes,ASD_traits
0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0
1,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,1
2,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1
3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1


# Data prep
Prepare the data and split data into testing and training

In [5]:
X= final_data.iloc[:,:-1]
y= final_data.iloc[:,-1]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.9,random_state=101)

# Models
Setup 3 Classification models to be ensembled later on

In [6]:
lgr=LogisticRegression()
lgr.fit(X_train,y_train)

rfc= RandomForestClassifier(n_estimators=500)
rfc.fit(X_train,y_train)

knn= KNeighborsClassifier(n_neighbors=105)
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=105)

# Results
Define estimators, setup ensemble system, train the ensemble system, and produce a confusion matrix and classification report

In [7]:
estimators=[('knn', knn), ('rfc', rfc), ('log_reg', lgr)]
ensemble = VotingClassifier(estimators, voting='hard')
ensemble.fit(X_train, y_train)
ensemblePred=ensemble.predict(X_test)

print(confusion_matrix(y_test,ensemblePred))
print(classification_report(y_test,ensemblePred))

[[ 11 296]
 [  6 636]]
              precision    recall  f1-score   support

           0       0.65      0.04      0.07       307
           1       0.68      0.99      0.81       642

    accuracy                           0.68       949
   macro avg       0.66      0.51      0.44       949
weighted avg       0.67      0.68      0.57       949

